In [28]:
import requests
import json
import pandas as pd
import time
from IPython.display import clear_output, display
import sys
import random
import numpy as np
import os
import psycopg2
# Strategy
# get list of players
# from list of players, get list of games
# from list of games, get game stats
from utils import apikey, payload, rate, regions
import utils
from sqlalchemy import create_engine

dbname = 'league'
engine = create_engine(f"postgresql+psycopg2://{utils.pgUsername}:{utils.pgPassword}@/{dbname}?host={utils.pgHost}")
#conn = psycopg2.connect(dbname=dbname,user=utils.pgUsername,password=utils.pgPassword,host=utils.pgHost)
#cursor = conn.cursor()

In [48]:
conn.commit()


In [50]:
conn.close()


In [126]:
def writeGameInit(pd_table, conn, keys, data_iter):
    """
    Execute SQL statement inserting data

    Parameters
    ----------
    table : pandas.io.sql.SQLTable
    conn : sqlalchemy.engine.Engine or sqlalchemy.engine.Connection
    keys : list of str
        Column names
    data_iter : Iterable that iterates the values to be inserted
    """    
    print(keys)
    # gets a DBAPI connection that can provide a cursor
    dbapi_conn = conn.connection
    with dbapi_conn.cursor() as cur:        
        query = f"INSERT INTO {pd_table.name} (gameid, gamecreation, queueid, platformid, region, tier) VALUES "
        vals = list()
        for data in data_iter:
            gameid = data[keys.index('gameid')]
            gamecreation = data[keys.index('timestamp')]
            queueid = data[keys.index('queue')]
            platformid = data[keys.index('platformid')]
            region = data[keys.index('region')]
            tier = data[keys.index('tier')]
            vals.append(f"({gameid}, {gamecreation}, {queueid}, '{platformid}', '{region}', '{tier}')")
        query += ", ".join(vals) + " ON CONFLICT ON CONSTRAINT games_gameid_key DO NOTHING;
        display(query)
        cur.execute(query)
        

In [127]:
df.to_sql('games_temp', engine,if_exists='append',method=writeGameInit)

['index', 'gameid', 'queue', 'timestamp', 'platformid', 'tier', 'region']


"INSERT INTO games_temp (gameid, gamecreation, queueid, platformid, region, tier) VALUES (3640364171, 1604096511411, 420, 'NA1', 'na1', 'GOLD'), (3640361136, 1604095205900, 420, 'NA1', 'na1', 'GOLD'), (3639806211, 1604053436032, 420, 'NA1', 'na1', 'GOLD'), (3638051956, 1603955769905, 420, 'NA1', 'na1', 'GOLD'), (3637954785, 1603952603352, 420, 'NA1', 'na1', 'GOLD'), (3637868432, 1603950046861, 420, 'NA1', 'na1', 'GOLD'), (3634845717, 1603781303647, 420, 'NA1', 'na1', 'GOLD'), (3634836839, 1603778527489, 420, 'NA1', 'na1', 'GOLD'), (3634791343, 1603775650292, 420, 'NA1', 'na1', 'GOLD'), (3633743604, 1603706136029, 420, 'NA1', 'na1', 'GOLD'), (3633770080, 1603703610938, 420, 'NA1', 'na1', 'GOLD'), (3632197251, 1603617214497, 420, 'NA1', 'na1', 'GOLD'), (3632241076, 1603614257080, 420, 'NA1', 'na1', 'GOLD'), (3632097606, 1603611868203, 420, 'NA1', 'na1', 'GOLD'), (3632100987, 1603609082146, 420, 'NA1', 'na1', 'GOLD'), (3632016040, 1603607229339, 420, 'NA1', 'na1', 'GOLD'), (3631958766, 16

In [33]:
region = 'na1'
tier = 'CHALLENGER'
version = 10
cursor = conn.cursor()
#def getSummoners(version=10):
#self.logger.info(f"Getting summoners who participated in games with version={version}")
# get list of games where game version is 10.*			
query = f"SELECT p1_summonerid, p2_summonerid, p3_summonerid, p4_summonerid, p5_summonerid, p6_summonerid,"\
        f"p7_summonerid, p8_summonerid, p9_summonerid, p10_summonerid FROM games WHERE region='{region}' AND tier='{tier}' and gameversion LIKE '{version}.%'"
results = None
cols = None
cursor.execute(query)
results =cursor.fetchall()
cols = [d[0] for d in cursor.description]

# get every summonerid that participated in those games
summonerids = list()
for r in results:
    for p in r:
        summonerids.append(p)
summonerids = list(set(summonerids))		

# get list of summoners that aren't in the database
query = f"SELECT * from summoners where region='{region}'"
results = None
cols = None
cursor.execute(query)
results = cursor.fetchall()
cols = [d[0] for d in cursor.description]
df = pd.DataFrame(results, columns=cols)
new_summoners = set(summonerids).difference(set(df['summonerid']))					
#self.logger.debug(f"new summoners: {new_summoners}")
print(new_summoners)
print("---------------------------------------------")
display(summonerids)
	

    


{'KUQFVAQG69I3AfO8WQpzmsTaStEJ9iyPEw9kjUvkQSjnl_o', 'zF1bhGcWKnLrbWs_gP9uxDwhKuuA2LMEMFIB7rEG1Ar3-pg', 'p_3Vv2zEJ1T51WlfOT7t5D4Ll43LRu053ZreV0pDqfo72-M', 'Losd76grQRjyv-s5EVoM-Pn3aHoluJDw7DnQylAxtXizXGY', 'teaJVlbfKfQsKyeiOyEsXaQa2XnMj7yzkknzhnf2YDjiL3E', 'Ndw3kcYaYVQ6ptuZh770CcTZrt01JDLWDPA6v0mk6ld7e6Hg', '0Cxr3az_C4bRO1gUpT5JLy_S8ixjGmlTwfpA1FQRsXER01ZF', 'cS6fKLvmZRN4sD9ktFdV_3PK9KznWk_E9kIqnsBBtujYx-3gyzGA6OiSUw', 'E9CstVZVziZybajBg4XyHCXDFiTWwOzrPTAOUVv3YmsPUTcR', 'o1fuk2BInnoFO3IoqubscZiANhmlCuWybn5UYXJ1U7T7U-jb', 'MqaKBPuQLN6CXcBkriBi1nQEN-_JzUZ0_X_TitHBpbw36VM', 'HphUO6qH_xVhM2Mf8g4hc7PJ-p9W0C02F3h80oH39FomYSm8', 'B39nc3SXkhCg3WZvw4v7gTH7szxv4tLr0feMvMD_KS-Ifr0', 'g9J90Y1Tc0stVTrYfLaDKLhdveHXqoKARW_-Er3CZPrTMklh', 'H2zkRZr_YlUWTnFDdFuvzVMPul34rgfEYyB08_mxUjTfHlms', '93vqT_36S4B3zrIJ0TeNNknJxVGRLuQNRCoD0Z-MCVCYdoE3', 'OInOkkYMQlg2am7NPcr-lTmv571fdxZonEdlySzK-SjFLmbr', 'OGBI8sGS8Ehqk9mHCaUd860_DmnIiJhwnRpYIotadv6SuZjz', 'YWJNt1AuioQTDIlGPSHFR6xCX-o1cjvX6A1SPzvm1OBzp9K0', 'SmshXP8

['U7d5Iw3xkzChNqCzKgT7q48ucGl6zDZvox0grlfiYjz5E1A',
 'o_F86UFlkfEXjFzRhrazTqiCCLDWcfY6ntrI6HEH-SSCjGxX',
 'wiu3zrRtosc9a-xEcDzwm4eDWV1Y8WoyO76KfWwHDR1DhWM',
 'C9XnLbK_n-HUhrU_zm5ot9KnlsCz1hDoFdA0nM-HFZHnZ7s',
 'Ao8f7gH4gOh2-l29cOWh_gw8cr58iFIhgls0CJgspEi8qSA',
 'bRez2ZcjFFOZV6afQzquu-hKy2S-8t1oxIFe3BRhruRlmA8',
 'rhzR9Zpp9GFBoRujiTPuSGSgsuScFyoQzazpDDloF5FTsuo',
 'wzLsSJzFC4GlnzNAgoXs5YCypKPJVGsfkC1AoZkfvke8-Ws',
 'tacrdioKoWoBq0Ob9jR8XLxm2FG6rtCpr_CSlgD05262RL6G',
 'nn9pN7pK3o6sMJgwQrLmHZuvoId-HY_xCdNqyWtY3sjD70g',
 'H5Zu5tuW_Dd7pLi7gn4TdFklTjilplhalRhSv8R2v7-mIOw',
 'owEfTP1wTjOa5Osh9_N0yQZi921SELf3bqLV2W70h7Ao3HU',
 'IWpJkWZsVam8esjJTrpjjSfN2ps4VxyAlF71EJAG0Uo',
 'jC90zMcE8VERepNAyyotSQa_9ZkhK9p1YImfu0eCobcOVGs',
 'kydEbY5kZ6V_VG4isPQ6Ilvzb8u97cN_gRUQ1W-HOly57K8',
 '9NZi3aP77pX0fz44Wdv-0YRdbpLOiYRI7OKarAsXIL1_Zgo',
 'mLk3_ga0flejPX5QcoTBxE3Ah853Y4NvRktuzkQwSTr50en8',
 'JvXsSznHjMQorwKr1kzIJ0oyzy0rOUxxmAG_qWAq3IayBYg',
 'KY7z_pd-X_upNm54J4dAGqu5OoNrLFmPpgq34a1C7cUNlgw',
 'PrDL-F1CdnY

In [38]:
#def writeSummoners(self, new_summoners):				
#self.logger.info(f"Writing {len(new_summoners)} new summoners to database")
tmp = list()
for s in new_summoners:
    tmp.append(f"('{s}','{region}','{tier}')")
insertQuery = "INSERT INTO summoners (summonerid, region, tier) VALUES " + ", ".join(tmp)
display(insertQuery)
cursor.execute(insertQuery)
conn.commit()
#self.logger.debug(f"writing: {insertQuery}")


"INSERT INTO summoners (summonerid, region, tier) VALUES ('KUQFVAQG69I3AfO8WQpzmsTaStEJ9iyPEw9kjUvkQSjnl_o','na1','CHALLENGER'), ('zF1bhGcWKnLrbWs_gP9uxDwhKuuA2LMEMFIB7rEG1Ar3-pg','na1','CHALLENGER'), ('p_3Vv2zEJ1T51WlfOT7t5D4Ll43LRu053ZreV0pDqfo72-M','na1','CHALLENGER'), ('Losd76grQRjyv-s5EVoM-Pn3aHoluJDw7DnQylAxtXizXGY','na1','CHALLENGER'), ('teaJVlbfKfQsKyeiOyEsXaQa2XnMj7yzkknzhnf2YDjiL3E','na1','CHALLENGER'), ('Ndw3kcYaYVQ6ptuZh770CcTZrt01JDLWDPA6v0mk6ld7e6Hg','na1','CHALLENGER'), ('0Cxr3az_C4bRO1gUpT5JLy_S8ixjGmlTwfpA1FQRsXER01ZF','na1','CHALLENGER'), ('cS6fKLvmZRN4sD9ktFdV_3PK9KznWk_E9kIqnsBBtujYx-3gyzGA6OiSUw','na1','CHALLENGER'), ('E9CstVZVziZybajBg4XyHCXDFiTWwOzrPTAOUVv3YmsPUTcR','na1','CHALLENGER'), ('o1fuk2BInnoFO3IoqubscZiANhmlCuWybn5UYXJ1U7T7U-jb','na1','CHALLENGER'), ('MqaKBPuQLN6CXcBkriBi1nQEN-_JzUZ0_X_TitHBpbw36VM','na1','CHALLENGER'), ('HphUO6qH_xVhM2Mf8g4hc7PJ-p9W0C02F3h80oH39FomYSm8','na1','CHALLENGER'), ('B39nc3SXkhCg3WZvw4v7gTH7szxv4tLr0feMvMD_KS-Ifr0','na1','CHALL

In [9]:
result = None
with engine.connect() as connection:
    query = f"SELECT * from summoners where region='{region}' and summonerid='DcqJLORFwEYSUuNlfxkSmNNVGX4DDAs3PQ-ffY8IixP3j33z'"
    result = connection.execute(query)
records = result.fetchall()
df = pd.DataFrame(records,columns=result.keys())

In [43]:
summonerId = summonerids[0]
query = f"SELECT * FROM summoners WHERE summonerid='{summonerId}' and region='{region}'"
cursor.execute(query)
result = cursor.fetchone()
cols = [d[0] for d in cursor.description]
print(cols)
print(result)

['summonerid', 'summonername', 'tier', 'rank', 'region', 'accountid', 'id', 'last_accessed', 'puuid']
('U7d5Iw3xkzChNqCzKgT7q48ucGl6zDZvox0grlfiYjz5E1A', 'Just Stibi', 'MASTER', 'I', 'na1', 'tIs5yJ0qgQAGM6VTjKVxXSps9Y_QqNIA5uFWNkBhnNrlRRspJGqSLeJj', 1446459, datetime.datetime(2020, 10, 26, 18, 13, 6, 967488), 'RibKMg3wdgbuKxlpJLVBxotp_C_iItU8cvXEqSpvH8tCxBjsyKJRgoRFsx20Dt2f88b6yy5Q7P54aA')


In [47]:
#def updateSummonerInfo(self, summonerId):
for summonerId in summonerids:
    
    query = f"SELECT * FROM summoners WHERE summonerid='{summonerId}' and region='{region}'"
    cursor.execute(query)
    result = cursor.fetchone()
    cols = [d[0] for d in cursor.description]
    puuidIdx = cols.index('puuid')
    if result[puuidIdx] == None:
        r = dict()
        req = f'https://{region}.api.riotgames.com/lol/summoner/v4/summoners/{summonerId}'
        resp = requests.get(req,headers=payload)
        time.sleep(1/rate)
        if resp.ok:
            data = json.loads(resp.content)
            r['puuid'] = data['puuid']
            r['summonerid'] = data['id']
            r['accountid'] = data['accountId']
            r['summonername'] = data['name']

        else:					
            if self.handleHTTPError(resp)==404:
                r['puuid'] = 404
                r['summonerid'] = summonerId

        # update the database
        updateQuery = f"UPDATE summoners SET puuid='{r['puuid']}', accountid='{r['accountid']}', summonername='{r['summonername']}' WHERE summonerid='{r['summonerid']}'"
        #print(updateQuery
        cursor.execute(updateQuery)
        conn.commit()

UPDATE summoners SET puuid='WXiyWklHfEfuGCgwEg8qNBwIZI4QePaHxmImh3abThul7JUs9w790xXNOwrBN2CcArxL1QRzIh_dNQ', accountid='tKAZ-2addFxgcqgwWNtbyZoGiTjJUaSP0_W8Fx9tXGtYvAcxTQeaSq5R', summonername='hua kai zai hui' WHERE summonerid='coK1l2RUqIJTWSFqFWP4SPwO5h6d95z1tSxvJxEJHC35Hm5K'
UPDATE summoners SET puuid='arAAaPEfET5owZLZBpOzq6xUGWU_jg96Rsl6ha5d7qzmy715TXk239ks-pkp6mhhPd9m7pJQmpFMAQ', accountid='ogYR8DAGgjsRPmq3T-ayfHOTR49nJMwpFCFn2g0O_IOA7Jvxwd4i5eOd', summonername='Avacago' WHERE summonerid='gI_XCxhwiTjMtaoMha_c3C8uz_dqp45gNV567mwHvk8lOvtx'
UPDATE summoners SET puuid='-nlZYKWDyAt1EYk-Eo43G8QPSJjUzEoFn-CWNiJyLEDmsPPd3EFiJ54FIFIgihke0P1_5a97GQjt4w', accountid='rzPjJY4BOvqgrKByjWUS_vR8vSU6vWBbQSZ7-BCOTyei6nwaJa35-RQJ', summonername='pensive emote' WHERE summonerid='IuLuope9piQQO8JSlHQrMB74QAMFV7BBI4WCR0BD-QFQWR2i'
UPDATE summoners SET puuid='lgX4SLqulhOJShS4PgJkSoN4_DZgkErUKpTHtt3EreAJOir_BNJc8wC5NJRdOgl2w0l_u9dl9FbNWQ', accountid='cN97d1zu4-TUSdmqzDP-i6tdPKLt691GruIib6pedgNhK83ieGCvr2Uz

KeyboardInterrupt: 

In [ ]:
#def getMatchList(self, summonerInfo):
# summonerInfo is a dict with at least the following keys:	
# 		summonerid, accountid, last_accessed
# returns a matchlist for this summoner as a pandas.DataFrame
self.logger.info(f"Getting matchlist for summoner {summonerInfo['summonerid']}")
# default to jan 1st 2020 to get only this year's games
jan01 = jan1=time.struct_time((2020,1,1,0,0,0,0,0,0))
jan01_timestamp = time.mktime(jan01)
last_accessed = jan01_timestamp
# but if we looked at this summoner before, then only get games up to the last time we accessed
if summonerInfo['last_accessed'] != None:
    last_accessed = summonerInfo['last_accessed'].timestamp()

df = pd.DataFrame()
beginIndex = 0
while True:
    resp, data = self.requestMatchlist(summonerInfo['accountid'], beginIndex=beginIndex)
    if not resp.ok:			
        self.logger.error(f"Error when requesting matches for summonerid='{summonerInfo['summonerid']}'")
        break;
    if data!=None or data['matches']==[]:
        self.logger.info(f"reached end of matches for {summonerInfo['summonerid']}")
        break;
    df = df.append(data['matches'], ignore_index=True)
    earliestTimestamp = df.iloc[-1]['timestamp']
    if earliestTimestamp/1000 > last_accessed:
        beginIndex+=100
    else:
        break;

# set all the column names to lowercase, then extract only the relevant columns
# then select only the games that happened after the timestamp threshold
if len(df)>0:
    df.columns= df.columns.str.strip().str.lower()
    df = df[['gameid','queue','timestamp','platformid']]
    df = df[df['timestamp']/1000>=last_accessed]
    df['tier'] = self.current_tier
    df['region'] = self.region
return df

In [55]:
#def requestMatchlist(self, accountId, beginIndex = None, endIndex = None):
accountId = 'lpOieAgIaWlp020ZktX-leWqpZBvQvMreMHw76TqNvnDTbN0nWwhevJA'
beginIndex = 0
endIndex = None
#self.logger.debug(f"Requesting matchlist for accountId={accountId}, beginIndex={beginIndex}")
extras = list()
extras.append("queue=420")
if beginIndex != None:
    extras.append(f"beginIndex={beginIndex}")
if endIndex != None:
    if endIndex > beginIndex and endIndex - beginIndex <= 100:
        extras.append(f"endIndex={endIndex}")
    else:
        self.logger.warning(f"endIndex must be greater than beginIndex and less than or equal to beginIndex+100, got beginIndex = {beginIndex} and endIndex = {endIndex}")				
req = f"https://{region}.api.riotgames.com/lol/match/v4/matchlists/by-account/{accountId}?"+"&".join(extras)
print(req)
resp = requests.get(req,headers=payload)
if resp.ok:
    data = json.loads(resp.content)
    #return resp, data
else:
    print(resp)
    #self.logger.debug("Error when requesting {req}")
    #self.handleHTTPError(resp)
    #return resp, None		
display(data)

https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/lpOieAgIaWlp020ZktX-leWqpZBvQvMreMHw76TqNvnDTbN0nWwhevJA?queue=420&beginIndex=0


{'matches': [{'platformId': 'NA1',
   'gameId': 3643912739,
   'champion': 117,
   'queue': 420,
   'season': 13,
   'timestamp': 1604293977926,
   'role': 'DUO_SUPPORT',
   'lane': 'BOTTOM'},
  {'platformId': 'NA1',
   'gameId': 3643854161,
   'champion': 117,
   'queue': 420,
   'season': 13,
   'timestamp': 1604291486891,
   'role': 'DUO_SUPPORT',
   'lane': 'BOTTOM'},
  {'platformId': 'NA1',
   'gameId': 3642568182,
   'champion': 267,
   'queue': 420,
   'season': 13,
   'timestamp': 1604213284557,
   'role': 'DUO_SUPPORT',
   'lane': 'NONE'},
  {'platformId': 'NA1',
   'gameId': 3642480894,
   'champion': 117,
   'queue': 420,
   'season': 13,
   'timestamp': 1604207253218,
   'role': 'DUO_SUPPORT',
   'lane': 'NONE'},
  {'platformId': 'NA1',
   'gameId': 3640623389,
   'champion': 117,
   'queue': 420,
   'season': 13,
   'timestamp': 1604110365871,
   'role': 'DUO_SUPPORT',
   'lane': 'BOTTOM'},
  {'platformId': 'NA1',
   'gameId': 3639426195,
   'champion': 117,
   'queue': 42

In [3]:
accountId = '24W6wGUaKWnHZD4JtybhAOIVnZdubjxPvlBkdhJaFQPknQI'
region = 'na1'
extras = list()
extras.append("queue=420")
extras.append("beginIndex=0")
req = f"https://{region}.api.riotgames.com/lol/match/v4/matchlists/by-account/{accountId}?"+"&".join(extras)
resp = requests.get(req, headers=payload)
data = json.loads(resp.content)

In [4]:
data

{'status': {'message': 'Bad Request - Expected message of type accountId, but found summonerId',
  'status_code': 400}}